### 导入包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### 导入数据并归一化

In [2]:
data=pd.read_csv('./clean-data.csv')
data=data.drop(['Unnamed: 0'],axis=1)
X=data.drop(['Group'],axis=1)
y=data['Group']
column=X.columns
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X=sc.fit_transform(X)
X=pd.DataFrame(data=X,columns=column)

### 划分训练集测试集

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=None)
print(f'Train shape : {X_train.shape}\nTest shape: {X_test.shape}')

Train shape : (132, 203)
Test shape: (33, 203)


In [4]:
#只在测试集上测试
def testpredict(estimator,Xtrain,ytrain,Xtest,ytest):
    print("只在测试集上测试")
    model=estimator.fit(Xtrain,ytrain)
    y_pred_train=model.predict(Xtrain)
    y_pred_test=model.predict(Xtest)
    confusion=confusion_matrix(ytest,y_pred_test)
    acc=accuracy_score(ytest, y_pred_test)
    rec=recall_score(ytest, y_pred_test)
    pre=precision_score(ytest, y_pred_test)
    f1=f1_score(ytest, y_pred_test)
    print('the confusion_matrix of the model is : \n',confusion)
    print('the accuracy of the model on testing set is ：',acc)
    print("the precision score of the model on testing set is : ",pre )
    print("the recall score of the model on testing set is :", rec)
    print('the f1_score of the model on testing set is :',f1)
    
    return acc,pre,rec,f1

In [5]:
#在整个数据集上交叉验证
def crosspredict(estimator,Xtrain,ytrain,cv):
    print("在整个数据集上交叉验证")
    y_pred_cross=cross_val_predict(estimator,Xtrain,ytrain,cv=cv)
    confusion_cross=confusion_matrix(ytrain,y_pred_cross)
    acc=accuracy_score(ytrain,y_pred_cross)
    pre=precision_score(ytrain, y_pred_cross)
    rec=recall_score(ytrain, y_pred_cross)
    f1=f1_score(ytrain, y_pred_cross)
    print('the confusion_matrix of the model is : \n',confusion_cross)
    print('the accuracy of the model is : ',acc)
    print("the precision score of the model is : ", pre)
    print("the recall score of the model is :", rec)
    print('the f1_score of the model  is :',f1)
    return acc,pre,rec,f1

### 特征提取

In [13]:
from sklearn.feature_selection import SequentialFeatureSelector
def searchNFeatures(estimator,nEnd = 11):
    n_feature_range = range(4,nEnd+1)
    featuremax2=0
    feature2=[]
    for n in n_feature_range:
        print("N features:", n)
        sel_seq = SequentialFeatureSelector(estimator=estimator, n_features_to_select=n)
        temp=sel_seq.fit(X, y)
        sel_seq_mask= X.columns[temp.get_support()]
        print('features:',sel_seq_mask)
        X_new=sel_seq.fit_transform(X,y)
        X_train1,X_test1,y_train1,y_test1 = train_test_split(X_new,y,test_size=0.2,random_state=42,stratify=None)
        
        acc2,pre2,rec2,f12=crosspredict(estimator,X_new,y,10)
        if f12>f1max2:
            featuremax2=n
            feature2=sel_seq_mask

    print('feature :',feature2)
    print('the best feature number is:',featuremax2)
    print("\n")

In [1]:
searchNFeatures(KNeighborsClassifier(n_neighbors=3, p=1),10)

feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010052M',
       'APG_0101010055P', 'APG_0101010176P'],
      dtype='object')
the best feature number is: 5

In [ ]:
searchNFeatures(SVC(C=1, coef0=10, degree=4, kernel='poly', probability=True),10)


feature : Index(['APG_0101010002M', 'APG_0101010010M', 'APG_0101010037P',
       'APG_0101010039M', 'APG_0101010055P', 'APG_0101010143P',
       'APG_0101010247P', 'APG_0101010300P', 'APG_0101010332P'],
      dtype='object')
the best feature number is: 9


In [ ]:
searchNFeatures(RandomForestClassifier(max_depth=8, n_estimators=105, n_jobs=-1,random_state=90),10)


feature : Index(['APG_0101010055P','APG_0101010262P','APG_0101010266P'],
      dtype='object')
the best feature number is: 3


In [ ]:
searchNFeatures(AdaBoostClassifier(learning_rate=0.1),10)


feature : Index(['APG_0101010033M', 'APG_0101010063M', 'APG_0101010067M',
       'APG_0101010147M', 'APG_0101010192M', 'APG_0101010236M'],
      dtype='object')
the best feature number is: 6



In [ ]:
searchNFeatures(LogisticRegression(C=0.1),10)


feature : Index(['APG_0101010010M', 'APG_0101010030M', 'APG_0101010041M',
       'APG_0101010043P', 'APG_0101010137P', 'APG_0101010150P',
       'APG_0101010152M', 'APG_0101010168P', 'APG_0101010176P',
       'APG_0101010356P'],
      dtype='object')
the best feature number is: 10


In [ ]:
searchNFeatures(XGBClassifier(booster='gblinear',learning_rate=0.3,n_estimators=10),10)


feature : Index(['APG_0101010010M','APG_0101010037P','APG_0101010063M',
'APG_0101010078P','APG_0101010150P','APG_0101010230P',
'APG_0101010279P','APG_0101010312P','APG_0101020013M'],
      dtype='object')
the best feature number is: 9


In [ ]:
searchNFeatures(lgb.LGBMClassifier(max_depth=3, n_estimators=300),10)


feature : Index(['APG_0101010036M', 'APG_0101010078P', 'APG_0101010130P',
       'APG_0101010192M', 'APG_0101010226P', 'APG_0101010242P',
       'APG_0101010265P'],
      dtype='object')
the best feature number is: 7


In [ ]:
searchNFeatures(DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=6),10)


feature : Index(['APG_0101010010M', 'APG_0101010055P', 'APG_0101010063M',
       'APG_0101010341M', 'APG_0101010354P'],
      dtype='object')
the best feature number is: 5
